## The Back Story...

Shakespeare ressurrected in the 21st century and needs to keep up with modern technology. On top of that, he has to find supplementary source of income since his playwrighting skills are no longer relevant in this day and age and in the midst of the cost of living crisis. He eventually found a job as a customer support specialist, but he still has difficulty understanding the modern English lingo. Equipped with some new tech skills he learned on the internet, Shakespeare now wants to explore Large Language Models to help him translate customer prompts in modern English into the English that he once knew...

### A sample customer prompt

In [37]:
customer_prompt = '''
Hey, I accidentally spilled oat milk on my toaster \
while making breakfast this morning. I unplugged it \
and waited for a few minutes before turning it back \
on, but now it's just making a weird buzzing noise \
and the indicator light doesn't switch on. Help me!
'''

### Shakespeare's chat prompt

In [38]:
style = 'like the plays of Shakespeare'

prompt = f'''Translate the text that is delimited by \
triple backticks into a style that is {style}. \
text: ```{customer_prompt}```
'''

## Part 1: Direct API calls to OpenAI

In [36]:
import openai
import os

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [39]:
def chat_with_openai(prompt, model='gpt-3.5-turbo'):
    message = {
        'role': 'user',
        'content': prompt,
    }
    # A lower temperature will make responses more deterministic
    response = openai.ChatCompletion.create(model=model,
                                            messages=[message],
                                            temperature=0)
    tokens = response['usage']['total_tokens']
    response = response['choices'][0]['message']['content']
    
    return response, tokens

In [40]:
response, tokens_used = chat_with_openai(prompt)

In [41]:
response

"Hark! 'Tis a tale of woe that I doth recount,\nForsooth, this morn whilst breaking fast, I erred.\nOat milk, by chance, did spill upon my toaster,\nAn accident, yet dire consequences ensued.\n\nSwiftly, I did unplug the device with care,\nAnd patiently, I waited for minutes three,\nEre I dared to rekindle its electric flame.\nAlas! A strange buzzing sound doth now persist,\nAnd the light, once bright, doth refuse to shine.\n\nPray, I beseech thee, lend me thy aid!\nForsooth, I am in need of thy wise counsel,\nTo mend this toaster, plagued by misfortune."

In [42]:
tokens_used

235

## Part 2: Using LangChain

LangChain gets your OpenAI API key in your `OPENAI_API_KEY` environment variable by default.

In [53]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [43]:
prompt_template = ChatPromptTemplate.from_template(prompt)

In [44]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="Translate the text that is delimited by triple backticks into a style that is like the plays of Shakespeare.\ntext: ```\nHey, I accidentally spilled oat milk on my toaster while making breakfast this morning. I unplugged it and waited for a few minutes before turning it back on, but now it's just making a weird buzzing noise and the indicator light doesn't switch on. Help me!\n```\n", template_format='f-string', validate_template=True)

In [50]:
chat_messages = prompt_template.format_messages(
                        style=style,
                        text=customer_prompt)

In [51]:
chat_messages[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is like the plays of Shakespeare.\ntext: ```\nHey, I accidentally spilled oat milk on my toaster while making breakfast this morning. I unplugged it and waited for a few minutes before turning it back on, but now it's just making a weird buzzing noise and the indicator light doesn't switch on. Help me!\n```\n", additional_kwargs={}, example=False)

In [54]:
chat = ChatOpenAI(temperature=0.0)
response = chat(chat_messages)

In [55]:
print(response.content)

Hark! 'Tis a tale of woe that I doth recount,
Forsooth, this morn whilst breaking fast in haste,
Mine hand didst falter, and oat milk didst spill,
Upon mine toaster, causing great distress.

Swiftly, I didst unplug the wretched device,
And bide my time, a few minutes didst pass,
Ere I dared to rekindle its fiery glow,
Yet, alas! A strange buzzing doth it emit,
And the light, once bright, doth refuse to shine.

Oh, I beseech thee, lend me thine aid,
Forsooth, what remedy canst thou proffer?
Pray, help me mend this toaster's sorry state,
That it may once again serve me with grace.


## Part 3: Using Output Parsers

Shakespeare's productivity at work soared after successfully using LLMs to help him understand their customers' English messages. His supervisor quickly saw this as an opportunity to assign him more "responsibility" - he now wants Shakespeare to read customer reviews of their products and extract valuable information from them so that they can gather insights about customer satisfaction for particular products.

In [73]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

### A sample customer review

In [61]:
customer_review = '''
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
'''

### Shakespeare's chat prompt

In [122]:
prompt = '''
For the following text, extract the following information: \

is_happy: Was the customer happy with their purchase? \
Answer Yes if they were. Answer No if they were not. \
Answer "Unknown" if the information is not found. \

delivery_days: How many days did it take the product to arrive? \
Answer -1 if the information is not found. \

price_value: Extract any phrases about the value or price of \
the product, and output them as a list. \

damages: Extract any phrases about damages that the customer \
found in the product, and output them as a list. \

Format the output as JSON with the following keys:
is_happy
delivery_days
price_value
damages

text: {text}
'''

### Getting the raw response without an Output Parser

In [123]:
prompt_template = ChatPromptTemplate.from_template(prompt)
print(prompt_template.messages[0].prompt)

input_variables=['text'] output_parser=None partial_variables={} template='\nFor the following text, extract the following information: \nis_happy: Was the customer happy with their purchase? Answer Yes if they were. Answer No if they were not. Answer "Unknown" if the information is not found. \ndelivery_days: How many days did it take the product to arrive? Answer -1 if the information is not found. \nprice_value: Extract any phrases about the value or price of the product, and output them as a list. \ndamages: Extract any phrases about damages that the customer found in the product, and output them as a list. \nFormat the output as JSON with the following keys:\nis_happy\ndelivery_days\nprice_value\ndamages\n\ntext: {text}\n' template_format='f-string' validate_template=True


In [124]:
chat_messages = prompt_template.format_messages(
                        text=customer_review)

In [125]:
chat_messages[0]

HumanMessage(content='\nFor the following text, extract the following information: \nis_happy: Was the customer happy with their purchase? Answer Yes if they were. Answer No if they were not. Answer "Unknown" if the information is not found. \ndelivery_days: How many days did it take the product to arrive? Answer -1 if the information is not found. \nprice_value: Extract any phrases about the value or price of the product, and output them as a list. \ndamages: Extract any phrases about damages that the customer found in the product, and output them as a list. \nFormat the output as JSON with the following keys:\nis_happy\ndelivery_days\nprice_value\ndamages\n\ntext: \nThis leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning

In [126]:
chat = ChatOpenAI(temperature=0.0)
response = chat(chat_messages)

In [127]:
print(response.content)

{
  "is_happy": "Yes",
  "delivery_days": 2,
  "price_value": ["slightly more expensive", "worth it for the extra features"],
  "damages": []
}


The API response looks like a json object, but when we print out the data type of the response content, it's actually a string. This makes it a bit difficult to parse the relevant data from the response.

In [128]:
print(type(response.content))

<class 'str'>


### Using an Output Parser to format the response

The output parser streamlines the process of parsing the necessary data from a chat response. This parser is useful when there are several outputs and we want to structure them properly for readability and future processing. It also allows us to specify the data types of each output.

In [74]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [129]:
is_happy = ResponseSchema(name='is_happy',
                          description='''Was the customer happy with their purchase?
                          Answer Yes if they were. Answer No if they were not.
                          Answer "Unknown" if the information is not found.
                          ''')
delivery_days = ResponseSchema(name='delivery_days',
                               description='''How many days did it take the product to arrive?
                               Answer -1 if the information is not found.
                               ''',
                               type='int')
price_value = ResponseSchema(name='price_value',
                             description='''Extract any phrases about the value or price of
                             the product, and output them as a list.
                             ''',
                             type='list')
damages = ResponseSchema(name='damages',
                             description='''Extract any phrases about damages that the customer
                             found in the product, and output them as a list.
                             ''',
                             type='list')

response_schemas = [
    is_happy,
    delivery_days,
    price_value,
    damages,
]

In [130]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [131]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"is_happy": string  // Was the customer happy with their purchase?
                          Answer Yes if they were. Answer No if they were not.
                          Answer "Unknown" if the information is not found.
                          
	"delivery_days": int  // How many days did it take the product to arrive?
                               Answer -1 if the information is not found.
                               
	"price_value": list  // Extract any phrases about the value or price of
                             the product, and output them as a list.
                             
	"damages": list  // Extract any phrases about damages that the customer
                             found in the product, and output them as a list.
                             
}
```


In [132]:
prompt = '''
For the following text, extract the following information: \

is_happy, delivery_days, price_value

Format the output as JSON with the following keys:
is_happy
delivery_days
price_value
damages

text: {text}

{format_instructions}
'''

In [133]:
prompt_template = ChatPromptTemplate.from_template(prompt)
print(prompt_template.messages[0].prompt)

input_variables=['format_instructions', 'text'] output_parser=None partial_variables={} template='\nFor the following text, extract the following information: \nis_happy, delivery_days, price_value\n\nFormat the output as JSON with the following keys:\nis_happy\ndelivery_days\nprice_value\ndamages\n\ntext: {text}\n\n{format_instructions}\n' template_format='f-string' validate_template=True


In [134]:
chat_messages = prompt_template.format_messages(
                        text=customer_review,
                        format_instructions=format_instructions)

In [135]:
print(chat_messages[0].content)


For the following text, extract the following information: 
is_happy, delivery_days, price_value

Format the output as JSON with the following keys:
is_happy
delivery_days
price_value
damages

text: 
This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"is_happy": string  // Was the customer happy with their purchase?
                          Answer Yes if they were. Answer No if they were not.
                          Answe

In [103]:
response = chat(chat_messages)

In [104]:
print(response.content)

```json
{
	"is_happy": "Yes",
	"delivery_days": 2,
	"price_value": ["slightly more expensive than the other leaf blowers out there", "worth it for the extra features"],
	"damages": []
}
```


In [105]:
output_dict = output_parser.parse(response.content)

In [106]:
print(output_dict)

{'is_happy': 'Yes', 'delivery_days': 2, 'price_value': ['slightly more expensive than the other leaf blowers out there', 'worth it for the extra features'], 'damages': []}


The output of the parser is a dictionary containing the various outputs that we have structured into dictionary keys. Each value is of the data type that we specified it to be in the format instructions.

In [107]:
type(output_dict)

dict

In [108]:
output_dict.get('is_happy')

'Yes'

In [109]:
output_dict.get('delivery_days')

2

In [110]:
output_dict.get('price_value')

['slightly more expensive than the other leaf blowers out there',
 'worth it for the extra features']

In [111]:
output_dict.get('damages')

[]